In [ ]:
# TRAINING

# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray

import os

from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.visualize import display_instances

from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image


from matplotlib import pyplot


import our_dataset

ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
images_path = '/home/test/data/nightowls/validation/nightowls_validation'
    
# train set
train_set = OurDataset()
train_set.load_dataset(images_path, ann_file_path, is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# val set
val_set = OurDataset()
val_set.load_dataset(images_path, ann_file_path, is_train=False)
val_set.prepare()
print('Validation: %d' % len(val_set.image_ids))

'''
k = 0
# plot first few images
for i in range(18,27):
    # define subplot
    pyplot.subplot(3,3,1+k)
    k = k+1
    # plot raw pixel data
    image = train_set.load_image(i)
    pyplot.imshow(image)
    # plot all masks
    mask, _ = train_set.load_mask(i)
    for j in range(mask.shape[2]):
        pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
        
# show the figure
pyplot.show()

#useful debug tool:
# enumerate all images in the dataset
i = 0
for image_id in train_set.image_ids:
    if i > 9:
        continue
  
    # load image info
    info = train_set.image_info[image_id]
    # display on the console
    print(info)
    i = i+1
'''

'''
# print all the images with the masks in the dataset
for i in range(len(train_set.image_ids)):
    # define image id
    image_id = i
    # load the image
    image = train_set.load_image(image_id)
    # load the masks and the class ids
    mask, class_ids = train_set.load_mask(image_id)
    # extract bounding boxes from the masks
    bbox = extract_bboxes(mask)
    # display image with masks and bounding boxes
    display_instances(image, bbox, mask, class_ids, train_set.class_names)
'''

from Mask_RCNN.mrcnn.config import Config
from Mask_RCNN.mrcnn.model import MaskRCNN


# define a configuration for the model
class KangarooConfig(Config):
	# Give the configuration a recognizable name
	NAME = "kangaroo_cfg"
	# Number of classes (background + kangaroo)
	NUM_CLASSES = 3 +1
	# Number of training steps per epoch
	STEPS_PER_EPOCH = 150

# prepare config
config = KangarooConfig()

# define the model
model = MaskRCNN(mode='training', model_dir='./trained_model', config=config)

# load weights (mscoco)
model.load_weights('/home/test/data/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# train weights (output layers or 'heads')
model.train(train_set, val_set, learning_rate=config.LEARNING_RATE, epochs=1, layers='heads')


In [ ]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray

import os

from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.visualize import display_instances

from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image


from matplotlib import pyplot
%matplotlib inline

from our_dataset import OurDataset

# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "kangaroo_cfg"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 3 +1
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
images_path = '/home/test/data/nightowls/validation/nightowls_validation'
    
# train set
train_set = OurDataset()
train_set.load_dataset(images_path, ann_file_path, is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# val set
val_set = OurDataset()
val_set.load_dataset(images_path, ann_file_path, is_train=False)
val_set.prepare()
print('Validation: %d' % len(val_set.image_ids))
    
    
# create config
cfg = PredictionConfig()

# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

# load model weights
model.load_weights('./trained_model/kangaroo_cfg20191028T1042/mask_rcnn_kangaroo_cfg_0001.h5', by_name=True)

from matplotlib.patches import Rectangle

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=1):
    # load image and mask
    for i in range(n_images):
        # load the image and mask
        image = dataset.load_image(i)
        mask, _ = dataset.load_mask(i)
        # convert pixel values (e.g. center)
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)[0]
        # define subplot
        
        fig = pyplot.figure(figsize=(150, 150))
        
        pyplot.subplot(n_images, 2, i*2+1)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Actual')
        # plot masks
        for j in range(mask.shape[2]):
            pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
        # get the context for drawing boxes
        pyplot.subplot(n_images, 2, i*2+2)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Predicted')
        ax = pyplot.gca()
        # plot each box
        for box in yhat['rois']:
            # get coordinates
            y1, x1, y2, x2 = box
            # calculate width and height of the box
            width, height = x2 - x1, y2 - y1
            # create the shape
            rect = Rectangle((x1, y1), width, height, fill=False, color='red')
            # draw the box
            ax.add_patch(rect)
    # show the figure
    pyplot.show()
    
    
# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg)
# plot predictions for test dataset
plot_actual_vs_predicted(val_set, model, cfg)

'''
# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on val dataset
test_mAP = evaluate_model(val_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)
'''

In [11]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray

import os

from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.visualize import display_instances

from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image


from matplotlib import pyplot
%matplotlib inline

from our_dataset import OurDataset

def fromOutputToAnn(image_id,out):
    
    print(out)
    
    for i in range(len(out['class_ids'])):
        class_id = out['class_ids'][i]
        bbox = out['rois'][i]
        score = out['scores'][i]
        
        entry = {
            "category_id": class_id,
            "bbox" : bbox,
            "score": score,
            "image_id" : image_id
        }
        print('entry ' + str(i))
        print(entry)
    

def generateAnnotations(dataset,model):
    i = 0
    for image_id in dataset.image_ids:
        if i > 2:
            continue

        # load image info
        info = train_set.image_info[image_id]
        # display on the console
        print(info)
        
        image = train_set.load_image(i)
        mask, _ = train_set.load_mask(i)
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)[0]
        
        fromOutputToAnn(info['real_id'],yhat)
        
        i = i+1
    

# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "kangaroo_cfg"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 3 +1
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
images_path = '/home/test/data/nightowls/validation/nightowls_validation'
    
# train set
train_set = OurDataset()
train_set.load_dataset(images_path, ann_file_path, is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

i = 0

for image_id in train_set.image_ids:
    if i > 9:
        continue
  
    # load image info
    info = train_set.image_info[image_id]
    # display on the console
    print(info)
    i = i+1
    
# create config
cfg = PredictionConfig()

# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

# load model weights
model.load_weights('./trained_model/kangaroo_cfg20191028T1042/mask_rcnn_kangaroo_cfg_0001.h5', by_name=True)

generateAnnotations(train_set,model)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Train: 150
{'id': 0, 'source': 'dataset', 'path': '/home/test/data/nightowls/validation/nightowls_validation/58c5812cbc260137e096966f.png', 'real_id': 7000000}
{'id': 1, 'source': 'dataset', 'path': '/home/test/data/nightowls/validation/nightowls_validation/58c5812cbc260137e0969672.png', 'real_id': 7000001}
{'id': 2, 'source': 'dataset', 'path': '/home/test/data/nightowls/validation/nightowls_validation/58c5812cbc260137e0969674.png', 'real_id': 7000002}
{'id': 3, 'source': 'dataset', 'path': '/home/test/data/nightowls/validation/nightowls_validation/58c5812cbc260137e0969675.png', 'real_id': 7000003}
{'id': 4, 'source': 'dataset', 'path': '/home/test/data/nightowls/validation/nightowls_validation/58c5812cbc260137e0969676.png', 'real_id': 7000004}
{'id': 5, 'source': 'dataset', 'path': '/home/test/data/nightowls/validation/nightowls_validation/58c5812cbc260137e0969677.png', 'real_id': 7000005}
{'id': 6, 's